In [25]:
import numpy as np
import pandas as pd
import pandas_profiling
from time import time

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
profile = pandas_profiling.ProfileReport(train_df)
profile.to_file(outputfile="./datareport.html")

#Save this for when we need to submit data back to kaggle
test_df_ids = test_df.loc[0:,"PassengerId"]
#drop data
train_df = train_df.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis = 1)
test_df = test_df.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis = 1)

#Add random seed so we get reproducible results.
seed = 7
np.random.seed(seed)
train_df.head(n=5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [4]:
train_df.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
train_df.count(), test_df.count()

(Survived    891
 Pclass      891
 Sex         891
 Age         714
 SibSp       891
 Parch       891
 Fare        891
 Embarked    889
 dtype: int64, Pclass      418
 Sex         418
 Age         332
 SibSp       418
 Parch       418
 Fare        417
 Embarked    418
 dtype: int64)

In [6]:
pandas_profiling.ProfileReport(train_df)


Number of variables,8
Number of observations,891
Total Missing (%),2.5%
Total size in memory,55.8 KiB
Average record size in memory,64.1 B
Numeric,5
Categorical,2
Boolean,1
Date,0
Text (Unique),0
Rejected,0


fill the missing data with the mean value

In [7]:
missing_values = ["Age", "Fare"]
for i in missing_values:
    mean = train_df.loc[:, i].mean()
    train_df.loc[:, i].fillna(mean, inplace =True)
    test_df.loc[:, i].fillna(mean, inplace =True)
#Non int case for embarked, only 2 values missing here so we drop them
train_df = train_df.dropna(axis = 'index')

In [8]:
train_df["Age"].head(n=6)

0    22.000000
1    38.000000
2    26.000000
3    35.000000
4    35.000000
5    29.699118
Name: Age, dtype: float64

In [9]:
train_df.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000
mean,0.382452,2.311586,29.653446,0.524184,0.382452,32.096681
std,0.486260,0.834700,12.968366,1.103705,0.806761,49.697504
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,22.000000,0.000000,0.000000,7.895800
50%,0.000000,3.000000,29.699118,0.000000,0.000000,14.454200
75%,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [10]:
train_df = pd.get_dummies(train_df, columns = ["Sex", "Pclass","Embarked"])
test_df = pd.get_dummies(test_df, columns = ["Sex", "Pclass","Embarked"])
train_df.head(n=5)

,Survived,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
0,0,22.0,1,0,7.2500,0,1,0,0,1,0,0,1
1,1,38.0,1,0,71.2833,1,0,1,0,0,1,0,0
2,1,26.0,0,0,7.9250,1,0,0,0,1,0,0,1
3,1,35.0,1,0,53.1000,1,0,1,0,0,0,0,1
4,0,35.0,0,0,8.0500,0,1,0,0,1,0,0,1


In [11]:
train_df.count()

Survived      889
Age           889
SibSp         889
Parch         889
Fare          889
Sex_female    889
Sex_male      889
Pclass_1      889
Pclass_2      889
Pclass_3      889
Embarked_C    889
Embarked_Q    889
Embarked_S    889
dtype: int64

In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

train, validate = train_test_split(train_df, test_size = 0.25, random_state = seed)
x_train = train.iloc[:,1:]
y_train = train.iloc[:,0]
x_val = validate.iloc[:,1:]
y_val = validate.iloc[:,0]

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(test_df)


In [13]:
from autosklearn.classification import AutoSklearnClassifier

clf = AutoSklearnClassifier()
print("Starting training..")
start = time()
clf.fit(x_train, y_train)
finish = time()
print("Finished training after {time} seconds. Now starting prediction inference".format(time = (finish-start)))
y_pred = clf.predict(x_val)
print("Finished")

[WARNING] [2018-03-08 12:12:40,042:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2018-03-08 12:12:40,042:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2018-03-08 12:12:40,360:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2018-03-08 12:12:40,360:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2018-03-08 12:12:40,795:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2018-03-08 12:12:40,795:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2018-03-08 12:12:42,276:smac.intensification.intensificatio

In [14]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

print(confusion_matrix(y_val, y_pred))
print(accuracy_score(y_val, y_pred))

[[117  17]
 [ 27  62]]
0.80269058296


In [16]:
clf.cv_results_

{'mean_fit_time': array([ 0.52006245,  0.78717566,  0.4028604 , ...,  1.39683461,
         0.49832463,  0.29798961]),
 'mean_test_score': array([ 0.80454545,  0.75909091,  0.82727273, ...,  0.82272727,
         0.73181818,  0.        ]),
 'param_balancing:strategy': masked_array(data = ['none' 'weighting' 'none' ..., 'none' 'none' 'none'],
              mask = [False False False ..., False False False],
        fill_value = N/A),
 'param_categorical_encoding:__choice__': masked_array(data = ['one_hot_encoding' 'one_hot_encoding' 'one_hot_encoding' ...,
  'no_encoding' 'no_encoding' 'no_encoding'],
              mask = [False False False ..., False False False],
        fill_value = N/A),
 'param_categorical_encoding:one_hot_encoding:minimum_fraction': masked_array(data = [0.01 0.010000000000000004 -- ..., -- -- --],
              mask = [False False  True ...,  True  True  True],
        fill_value = 1e+20),
 'param_categorical_encoding:one_hot_encoding:use_minimum_fraction': masked_ar

In [71]:
print(clf.show_models())
print(clf.sprint_statistics())

[(0.380000, SimpleClassificationPipeline({'balancing:strategy': 'weighting', 'categorical_encoding:__choice__': 'no_encoding', 'classifier:__choice__': 'qda', 'imputation:strategy': 'median', 'preprocessor:__choice__': 'extra_trees_preproc_for_classification', 'rescaling:__choice__': 'normalize', 'classifier:qda:reg_param': 0.1811885878554167, 'preprocessor:extra_trees_preproc_for_classification:bootstrap': 'True', 'preprocessor:extra_trees_preproc_for_classification:criterion': 'entropy', 'preprocessor:extra_trees_preproc_for_classification:max_depth': 'None', 'preprocessor:extra_trees_preproc_for_classification:max_features': 0.20798735059489348, 'preprocessor:extra_trees_preproc_for_classification:max_leaf_nodes': 'None', 'preprocessor:extra_trees_preproc_for_classification:min_impurity_decrease': 0.0, 'preprocessor:extra_trees_preproc_for_classification:min_samples_leaf': 16, 'preprocessor:extra_trees_preproc_for_classification:min_samples_split': 18, 'preprocessor:extra_trees_prep

In [20]:
predictions = clf.predict(x_test)
predictions

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0,

In [22]:
data_to_submit = pd.DataFrame({
    'PassengerId':test_df_ids,
    'Survived':predictions
})
data_to_submit.to_csv("./results/autosk_results.csv", index=False)

And the results on kaggle are: 78.94%
Now lets see the results when we let autosk preprocess the data.

In [75]:
#Start clean
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

#Save this for when we need to submit data back to kaggle
test_df_ids = test_df.loc[0:,"PassengerId"]

train_df = train_df.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis = 1)
test_df = test_df.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis = 1)
train_df = pd.get_dummies(train_df, columns = ["Sex", "Pclass","Embarked"])
test_df = pd.get_dummies(test_df, columns = ["Sex", "Pclass","Embarked"])

train, validate = train_test_split(train_df, test_size = 0.25, random_state = seed)
x_train = train.iloc[:,1:]
y_train = train.iloc[:,0]
x_val = validate.iloc[:,1:]
y_val = validate.iloc[:,0]

In [76]:
x_train.head(n=1)

,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
709,NaN,1,1,15.2458,0,1,0,0,1,1,0,0


In [80]:
newclf = AutoSklearnClassifier(time_left_for_this_task=3600)
print("Starting training..")
start = time()
newclf.fit(x_train, y_train)
finish = time()
print("Finished training after {time} seconds. Now starting prediction inference".format(time = (finish-start)))
y_pred = newclf.predict(x_val)
print("Finished")

Starting training..
Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (35.513893)
[WARNING] [2018-03-08 15:11:16,530:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2018-03-08 15:11:16,530:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
Finished training after 30.551493644714355 seconds. Now starting prediction inference
Finished


In [81]:
print(confusion_matrix(y_val, y_pred))
print(accuracy_score(y_val, y_pred))

[[114  21]
 [ 33  55]]
0.757847533632


In [79]:
newpredictions = clf.predict(x_test)
data_to_submit = pd.DataFrame({
    'PassengerId':test_df_ids,
    'Survived':newpredictions
})
data_to_submit.to_csv("./results/autosk_results_without_preprocessing.csv", index=False)